In [1]:
from IPython.core.events import EventManager
from IPython import get_ipython
import time

global_start_time = None

def start_timer(event):
    global global_start_time
    global_start_time = time.time()

def stop_timer(event):
    global global_start_time
    if global_start_time:
        duration = time.time() - global_start_time
        print(f"Cell execution time: {duration:.3f} seconds")
        global_start_time = None

ip = get_ipython()
ip.events.register('pre_run_cell', start_timer)
ip.events.register('post_run_cell', stop_timer)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from skimage import io 
from shapely.geometry import Polygon
from matplotlib.patches import Polygon as mplPolygon
from glob import glob
import cv2
from scipy import ndimage
import matplotlib.colors as mcolors
from scipy.spatial import KDTree
from sklearn.neighbors import NearestNeighbors
from scipy import ndimage as ndi
from scipy.ndimage import binary_dilation

Cell execution time: 1.940 seconds


In [3]:
df = pd.read_excel("../data/geomx_221_223.xlsx",sheet_name="SegmentProperties")
df = df.loc[df["SlideName"]=="1 PTB-22.1"]
df = df[["ROILabel","ROICoordinateX","ROICoordinateY","AOISurfaceArea"]]

Cell execution time: 0.975 seconds


In [4]:
mat = io.imread("../img/1 PTB-22.1.tiff")

Cell execution time: 0.280 seconds


In [5]:
# adjustments between metadata coordinates and approximate annotation coordinates 
x_221_sf = (11008-10172)/(42949.5-39599)
y_221_sf = (3569-1523)/(19294.5-11100)
x_221_cnst = 10172-39599*x_221_sf
y_221_cnst = 1523-11100*y_221_sf

Cell execution time: 0.000 seconds


In [6]:
comp = glob("../composite/PTB221*")
comp = {int(x.split("_")[-1].split(".png")[0]):x for x in comp}

Cell execution time: 0.002 seconds


In [7]:
xw =128

for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=np.round((x*x_221_sf)+x_221_cnst).astype(int)
    y=np.round((y*y_221_sf)+y_221_cnst).astype(int)
    xw =128
    
    img_bb = mat[(x-xw):(x+xw),(y-xw):(y+xw)]

    # bounding box is white 
    white_mask = (img_bb == [255,255,255,255]).all(-1)
    white_pixels = np.where(white_mask)
    
    # find the closest point to this middle 
    starting_point = np.argmin(np.abs(np.array(white_pixels).T-np.array([img_bb.shape[1]/2,img_bb.shape[0]/2])).sum(1))
    # this middle point is then used as the basis for flooding the mask layer 
    mid_point = (white_pixels[1][starting_point],white_pixels[0][starting_point])

    dilation_iter = 1

    # 19 and 20 are in contact, so we cannot flood their outlines 
    if roi in [19,20]:
        flood_mask = white_mask.astype(np.uint8).copy()
        _, flood_mask, _, _ = cv2.floodFill(flood_mask, None, (int(img_bb.shape[1]/2),int(img_bb.shape[0]/2)), 2)
        interior_mask = (flood_mask == 2)
        dilation_iter = 3
    else:
        flood_mask = ~white_mask.astype(np.uint8).copy()
        _, flood_mask, _, _ = cv2.floodFill(flood_mask, None, mid_point, 2)
        interior_mask = (flood_mask == 2)
    
    image = (~(binary_dilation(interior_mask, structure=np.ones((3,3)), iterations = dilation_iter) & white_mask)).astype(np.uint8)*255
    target_image = cv2.imread(comp[roi], cv2.IMREAD_UNCHANGED) 

    smooth_image = ndimage.gaussian_filter(image, sigma=3)
    resized_smooth_image = cv2.resize(smooth_image, (target_image.shape[1], target_image.shape[0]))
    mask = ~(resized_smooth_image>200)

    # mask has two colors, empty and white mask with low alpha 
    colors = [(0, 0, 0, 0), (1, 1, 1, 0.25)] 
    cmap = mcolors.LinearSegmentedColormap.from_list('my_cmap', colors, N=2)
    
    gaus_mask = ndimage.gaussian_filter(mask, sigma=3)
    dilate_mask = cv2.dilate(gaus_mask.astype(np.uint8), None, iterations=3)
    final_mask = cv2.erode(dilate_mask.astype(np.uint8), None, iterations=9)
    
    np.save("../bb_mask/"+"PTB221_"+(str(roi))+".npy",final_mask)

    # bounding box + composite 
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(cv2.cvtColor(target_image, cv2.COLOR_BGRA2RGBA))
    plt.imshow(final_mask,cmap=cmap)
    plt.axis("off")
    plt.savefig("../bb_comp/"+"PTB221_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()

    # original annotation bounding box
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(img_bb)
    plt.axis("off")
    plt.savefig("../bb_annot/"+"PTB221_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()

    # cut out of composite within bounding box 
    filled_mask = ndi.binary_fill_holes(final_mask)
    masked_image = np.zeros_like(target_image,)
    masked_image[filled_mask == 1] = target_image[filled_mask == 1]

    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGRA2RGBA))
    plt.axis("off")
    plt.savefig("../bb_comp_mask/"+"PTB221_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()

<Figure size 640x480 with 0 Axes>

Cell execution time: 227.917 seconds


In [8]:
df = pd.read_excel("../data/geomx_221_223.xlsx",sheet_name="SegmentProperties")
df = df.loc[df["SlideName"]=="2 PTB-22.3"]
df = df[["ROILabel","ROICoordinateX","ROICoordinateY","AOISurfaceArea"]]

Cell execution time: 1.365 seconds


In [9]:
mat = io.imread("../img/2 PTB-22.3-2.tiff")

Cell execution time: 0.265 seconds


In [10]:
# adjustments between metadata coordinates and approximate annotation coordinates 
x_223_sf = (4318-6234)/(16205.8-23872)
y_223_sf = (3344-5892)/(18429.5-28631)
x_223_cnst = 4318-16205.8*x_223_sf
y_223_cnst = 3344-18429.5*y_223_sf

Cell execution time: 0.000 seconds


In [11]:
comp = glob("../composite/PTB223*")
comp = {int(x.split("_")[-1].split(".png")[0]):x for x in comp}

Cell execution time: 0.001 seconds


In [12]:
for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=np.round((x*x_223_sf)+x_223_cnst).astype(int)
    y=np.round((y*y_223_sf)+y_223_cnst).astype(int)
    xw =128
    
    img_bb = mat[(x-xw):(x+xw),(y-xw):(y+xw)]

    white_mask = (img_bb == [255,255,255,255]).all(-1)
    white_pixels = np.where(white_mask)

    # 8 has an origin lower than its middle where a separate box is defaulted, hence we change the starting point
    if roi in [8]:
        starting_point = np.argmin(np.abs(np.array(white_pixels).T-np.array([img_bb.shape[1]/2,img_bb.shape[0]])).sum(1))
    else:
        starting_point = np.argmin(np.abs(np.array(white_pixels).T-np.array([img_bb.shape[1]/2,img_bb.shape[0]/2])).sum(1))

    mid_point = (white_pixels[1][starting_point],white_pixels[0][starting_point])

    # we want to flood the outline 
    flood_mask = ~white_mask.astype(np.uint8).copy()
    _, flood_mask, _, _ = cv2.floodFill(flood_mask, None, mid_point, 2)
    interior_mask = (flood_mask == 2)

    image = (~(binary_dilation(interior_mask, structure=np.ones((3,3)), iterations = 1) & white_mask)).astype(np.uint8)*255
    target_image = cv2.imread(comp[roi], cv2.IMREAD_UNCHANGED) 

    smooth_image = ndimage.gaussian_filter(image, sigma=3)
    resized_smooth_image = cv2.resize(smooth_image, (target_image.shape[1], target_image.shape[0]))
    mask = ~(resized_smooth_image>200)

    # mask has two colors, empty and white mask with low alpha 
    colors = [(0, 0, 0, 0), (1, 1, 1, 0.25)] 
    cmap = mcolors.LinearSegmentedColormap.from_list('my_cmap', colors, N=2)
    
    gaus_mask = ndimage.gaussian_filter(mask, sigma=3)
    dilate_mask = cv2.dilate(gaus_mask.astype(np.uint8), None, iterations=3)
    final_mask = cv2.erode(dilate_mask.astype(np.uint8), None, iterations=9)
    
    np.save("../bb_mask/"+"PTB223_"+(str(roi))+".npy",final_mask)

    # bounding box + composite 
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(cv2.cvtColor(target_image, cv2.COLOR_BGRA2RGBA))
    plt.imshow(final_mask,cmap=cmap)
    plt.axis("off")
    plt.savefig("../bb_comp/"+"PTB223_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()

    # original annotation bounding box
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(img_bb)
    plt.axis("off")
    plt.savefig("../bb_annot/"+"PTB223_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()

    # cut out of composite within bounding box 
    filled_mask = ndi.binary_fill_holes(final_mask)
    masked_image = np.zeros_like(target_image,)
    masked_image[filled_mask == 1] = target_image[filled_mask == 1]
    
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGRA2RGBA))
    plt.axis("off")
    plt.savefig("../bb_comp_mask/"+"PTB223_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()

<Figure size 640x480 with 0 Axes>

Cell execution time: 131.071 seconds


In [13]:
# PTB 22.2 already has high quality individual bounding boxes per spot 
df = pd.read_excel("../data/geomx_222.xlsx",sheet_name="SegmentProperties")
df = df.loc[df["SlideName"]=="PTB22.2"]
df = df[["ROILabel","ROICoordinateX","ROICoordinateY","AOISurfaceArea"]]

Cell execution time: 1.112 seconds


/Users/jperrie/miniconda3/envs/scvi/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [14]:
comp = glob("../composite/PTB222*")
comp = {int(x.split("_")[-1].split(".png")[0]):x for x in comp}

Cell execution time: 0.001 seconds


In [15]:
for spot in df.iterrows():

    roi=spot[1][["ROILabel"]].astype(int).values[0]
    img_bb = io.imread("../bb_annot/PTB222_"+str(roi)+".png")
    img_bb = img_bb[:img_bb.shape[1],:img_bb.shape[1]]
    pad = 0
    img_bb = img_bb[pad:img_bb.shape[1]-pad,pad:img_bb.shape[0]-pad]

    white_mask = (img_bb == [255,255,255,255]).all(-1)
    white_pixels = np.where(white_mask)
    
    starting_point = np.argmin(np.abs(np.array(white_pixels).T-np.array([img_bb.shape[1]/2,img_bb.shape[0]/2])).sum(1))
    mid_point = (white_pixels[1][starting_point],white_pixels[0][starting_point])

    # we want to flood the outline 
    flood_mask = ~white_mask.astype(np.uint8).copy()
    _, flood_mask, _, _ = cv2.floodFill(flood_mask, None, mid_point, 2)
    interior_mask = (flood_mask == 2)

    image = (~(binary_dilation(interior_mask, structure=np.ones((3,3)), iterations = 9) & white_mask)).astype(np.uint8)*255
    target_image = cv2.imread(comp[roi], cv2.IMREAD_UNCHANGED) 

    smooth_image = ndimage.gaussian_filter(image, sigma=3)
    resized_smooth_image = cv2.resize(smooth_image, (target_image.shape[1], target_image.shape[0]))
    mask = ~(resized_smooth_image>200)

    # mask has two colors, empty and white mask with low alpha 
    colors = [(0, 0, 0, 0), (1, 1, 1, 0.25)] 
    cmap = mcolors.LinearSegmentedColormap.from_list('my_cmap', colors, N=2)

    # more dilation because bounding box annotation is thin 
    # also reorder with dilation first 
    dilate_mask = cv2.dilate(mask.astype(np.uint8), None, iterations=15)
    gaus_mask = ndimage.gaussian_filter(dilate_mask, sigma=1)
    final_mask = cv2.erode(gaus_mask.astype(np.uint8), None, iterations=3)

    np.save("../bb_mask/"+"PTB222_"+(str(roi))+".npy",final_mask)

    # bounding box + composite 
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(cv2.cvtColor(target_image, cv2.COLOR_BGRA2RGBA))
    plt.imshow(final_mask,cmap=cmap)
    plt.axis("off")
    plt.savefig("../bb_comp/"+"PTB222_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()

    # cut out of composite within bounding box 
    filled_mask = ndi.binary_fill_holes(final_mask)
    masked_image = np.zeros_like(target_image,)
    masked_image[filled_mask == 1] = target_image[filled_mask == 1]
    
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGRA2RGBA))
    plt.axis("off")
    plt.savefig("../bb_comp_mask/"+"PTB222_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()

<Figure size 640x480 with 0 Axes>

Cell execution time: 198.126 seconds


In [16]:
# PTB 21.1 already has high quality individual bounding boxes per spot 
df = pd.read_excel("../data/geomx_211.xlsx",sheet_name="SegmentProperties")
df = df.loc[df["SlideName"]=="B172914-2"]
df = df[["ROILabel","ROICoordinateX","ROICoordinateY","AOISurfaceArea"]]

Cell execution time: 0.868 seconds


In [17]:
comp = glob("../composite/PTB211*")
comp = {int(x.split("_")[-1].split(".png")[0]):x for x in comp}

Cell execution time: 0.001 seconds


In [18]:
for spot in df.iterrows():
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    img_bb = io.imread("../bb_annot/PTB211_"+str(roi)+".png")
    img_bb = img_bb[:img_bb.shape[1],:img_bb.shape[1]]
    pad = 0
    img_bb = img_bb[pad:img_bb.shape[1]-pad,pad:img_bb.shape[0]-pad]

    # find all white pixels using less strict check 
    condition1 = np.all(img_bb == [255, 255, 255, 255], axis=-1)
    condition2 = np.all(img_bb == [254, 254, 254, 255], axis=-1)
    condition3 = np.all(img_bb == [254, 255, 255, 255], axis=-1)
    condition4 = np.all(img_bb == [255, 254, 255, 255], axis=-1)
    condition5 = np.all(img_bb == [255, 255, 254, 255], axis=-1)
    white_mask = np.logical_or(np.logical_or(np.logical_or(np.logical_or(condition1, condition2),condition3),condition4),condition5)

    white_pixels = np.where(white_mask)
    
    starting_point = np.argmin(np.abs(np.array(white_pixels).T-np.array([img_bb.shape[1]/2,img_bb.shape[0]/2])).sum(1))
    mid_point = (white_pixels[1][starting_point],white_pixels[0][starting_point])

    # we want to flood the outline 
    flood_mask = ~white_mask.astype(np.uint8).copy()
    _, flood_mask, _, _ = cv2.floodFill(flood_mask, None, mid_point, 2)
    interior_mask = (flood_mask == 2)

    image = (~(binary_dilation(interior_mask, structure=np.ones((3,3)), iterations = 9) & white_mask)).astype(np.uint8)*255
    target_image = cv2.imread(comp[roi], cv2.IMREAD_UNCHANGED) 

    smooth_image = ndimage.gaussian_filter(image, sigma=3)
    resized_smooth_image = cv2.resize(smooth_image, (target_image.shape[1], target_image.shape[0]))
    mask = ~(resized_smooth_image>200)

    # mask has two colors, empty and white mask with low alpha 
    colors = [(0, 0, 0, 0), (1, 1, 1, 0.25)] 
    cmap = mcolors.LinearSegmentedColormap.from_list('my_cmap', colors, N=2)

    # more dilation because bounding box annotation is thin 
    # also reorder with dilation first 
    dilate_mask = cv2.dilate(mask.astype(np.uint8), None, iterations=15)
    gaus_mask = ndimage.gaussian_filter(dilate_mask, sigma=1)
    final_mask = cv2.erode(gaus_mask.astype(np.uint8), None, iterations=3)

    np.save("../bb_mask/"+"PTB211_"+(str(roi))+".npy",final_mask)

    # bounding box + composite 
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(cv2.cvtColor(target_image, cv2.COLOR_BGRA2RGBA))
    plt.imshow(final_mask,cmap=cmap)
    plt.axis("off")
    plt.savefig("../bb_comp/"+"PTB211_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()
    
    filled_mask = ndi.binary_fill_holes(final_mask)
    masked_image = np.zeros_like(target_image,)
    masked_image[filled_mask == 1] = target_image[filled_mask == 1]

    # cut out of composite within bounding box 
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGRA2RGBA))
    plt.axis("off")
    plt.savefig("../bb_comp_mask/"+"PTB211_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()

<Figure size 640x480 with 0 Axes>

Cell execution time: 185.800 seconds
